# 4주차 : 

In [1]:
# HuggingFace Transformers 라이브러리를 설치합니다.
!pip install -q transformers

     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 46.1 MB/s 
     |████████████████████████████████| 163 kB 70.5 MB/s 


In [2]:
from transformers import AutoModelForSequenceClassification, Trainer, AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
id2label = {0: '날씨 묻기', 1: '관광지 추천', 2: '숙소 추천', 3: '맛집 추천', 4: '인사', 5: '소개', 6:'활동 추천', 7:'카페 추천'}

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [3]:
# 저장한 모델을 불러오기 위해 구글 드라이브에 마운트합니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# 모델 불러오기 (경로 수정 필요 -> 모델 관련 파일들을 모아놓은 폴더의 경로)
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/챗봇 프로젝트/4주차/06/my_model", local_files_only=True)
trainer = Trainer(model=model)

In [6]:
def inference(text):
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)
  logits = outputs.logits

  probs = logits.squeeze().cpu()
  predictions = np.where(probs==probs.max(), 1, 0)
  predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

  return predicted_labels[0]

In [7]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

Completed!


In [8]:
# 다음의 명령어로 ngrok linux version 을 설치

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2022-10-01 09:09:25--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13770165 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.13M  53.3MB/s    in 0.2s    

2022-10-01 09:09:26 (53.3 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13770165/13770165]



In [9]:
# 압축 해제

!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [30]:
# ngrok 사이트 (https://dashboard.ngrok.com/get-started/setup)에서 회원가입을 한뒤
# "Your Authtoken" 탭을 클릭하여 authtoken을 복사하여 붙여넣습니다.

!./ngrok authtoken 2AobOWreg1Cj5sYHFJ4rCPyQBt4_6HVVnb8nMQ4MFvKRzePCi
# 2FCXs61JYSF9rY3E6Grrhbzsaws_4UZTVN8zBssaE4fKFCYTM

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [22]:
# 플라스크 임포트
from flask import Flask
from flask import render_template, request
from flask_ngrok import run_with_ngrok

In [23]:
# softmax 와 threshold 를 사용하는 새로운 인퍼런스 함수
import torch
import torch.nn.functional as F

def inference_with_softmax(text):
  encoding = tokenizer(text, return_tensors = 'pt')
  encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

  outputs = trainer.model(**encoding)

  logits = outputs.logits

  probs = logits.squeeze().cpu()
  softmax_probs = F.softmax(probs, dim=-1)
  labels_predict = np.where(softmax_probs==softmax_probs.max())[0][0]

  if softmax_probs[labels_predict] < 0.97:
    return '기타'
  else:
    return id2label[labels_predict]

In [24]:
import random

def answer_sentence(pred):

  if pred == '날씨 묻기':

      date= ['오늘', '내일', '어제', '그저께', '내일 모레']
      choice_date = random.choice(date)

      weather_2 = ['추워요.', '따뜻해요.', '조금 쌀쌀해요.', '비가 올 것 같아요.' ,'태풍이 와요.', '바람이 불어요.', '더워요.']
      choice_weather_2= random.choice(weather_2)

      attractions_2= ['제주시', '서귀포시', '애월읍', '안덕면', '성산읍', '조천읍', '구좌읍', '한림읍', '남원읍']
      choice_attractions_2 = random.choice(attractions_2)

      sentence = f'{choice_date}, {choice_attractions_2}의 날씨는 {choice_weather_2}'

  elif pred == '관광지 추천':
 
    attraction = ['쇠소깍', '용머리해안', '함덕 해수욕장', '우도']
    choice_attraction = random.choice(attraction)
 
    nice = ['경치 좋은', '바다가 예쁜', '단체로 갈 만한', '사진찍기 좋은']
    choice_nice = random.choice(nice)
 
    sentence = f'제주 {choice_nice}, {choice_attraction} 가 보세요!'


  elif pred == '숙소 추천':
    accomodation = ['베니키아 호텔', '위드 스테이 호텔 제주', '제주 라온 호텔', '신신호텔 천지연']
    choice_accomodation = random.choice(accomodation)

    location = ['애월에 있는', '제주시에 있는', '한림에 있는', '서귀포에 있는']
    choice_location = random.choice(location)

    sentence = f'제주 {choice_location}, {choice_accomodation} 가 보시는 건 어떠세요?'
  
  elif pred == '맛집 추천':

    restaurant = ['숙성도', '빵귿', '살찐 고등어', '복집식당']
    choice_restaurant = random.choice(restaurant)

    delicious = ['삼겹살이 맛있는', '빵이 맛있는', '고등어가 유명한', '갈치구이 유명한']
    choice_delicious = random.choice(delicious)
    sentence = f'제주 {choice_delicious}, {choice_restaurant} 어떠세요?'
  
  elif pred=='인사':
    greeting = ['안녕하세요','혼저 옵서예','안녕하우꽈','옵데강, 혼저오십서.']
    choice_greeting = random.choice(greeting)

    closing = ['궁금한것을 알려드려요~!','궁금한것을 물어보세요~!','무엇이든 물어보세요~!']
    choice_closing = random.choice(closing)

    sentence = f'{choice_greeting} 저는 제주 관광 챗봇 입니다. {choice_closing}'

  elif pred == '소개':
    sentence = '저는 제주도의 <날씨>,<숙소>,<맛집>,<활동>,<카페> 기능에 관하여 이용이 가능합니다.'

  elif pred == "활동 추천":

    act_1 = ["인기 많은","즐거운","재밌는"]
    choice_act_1 = random.choice(act_1)

    act_2 =["둘레길 걷기","오름 방문","바닷길 드라이브"]
    choice_act_2 = random.choice(act_2)
    sentence = f"{choice_act_1}, {choice_act_2} 해보는 건 어때요?"

  elif pred == '카페 추천':
    cafe=['제주특별자치도 제주시 한경면 낙원로 32 에 있는 산노루','제주특별자치도 서귀포시 이어도로1027번길 34에 있는 카페 귤 꽃다락','제주특별자치도 서귀포시 516로 717에 있는 서귀다원','제주특별자치도 제주시 은수길 110 2F에 있는 우연못']
    choice_cafe = random.choice(cafe)
    ending=['어떠세요?','가보세요~!','가보는걸 추천 드립니다!','추천 합니다.']
    choice_ending=random.choice(ending)
    sentence = f'{choice_cafe},{choice_ending}'
  
  else:
    sentence = "죄송한데, 무슨 말씀인지 잘 모르겠어요."
  
  return sentence

In [25]:
# 입력 : 사용자가 입력한 텍스트
# 출력 : 의도에 대한 답변

def answer(text):
  
  pred = inference_with_softmax(text)
  # pred = inference(text)
  
  # id2label.values() : dict_values(['날씨 묻기', '관광지 추천', '숙소 추천', '맛집 추천', '인사', '소개', '활동 추천', '카페 추천'])
  if pred in id2label.values():
    return (True, answer_sentence(pred))
  else:
    return (False, answer_sentence(pred))

In [26]:
def answer_system(input_text):
  
  print("안녕하세요, 무엇을 도와드릴까요?")
  _, sentence = answer(input_text)
  
  return sentence 

In [ ]:
from flask import Flask, url_for, redirect, render_template, request

TEMPLATE_FOLDER = '/content/drive/MyDrive/Colab Notebooks/챗봇 프로젝트/4주차/06/templates'
STATIC_FOLDER = '/content/drive/MyDrive/Colab Notebooks/챗봇 프로젝트/4주차/06/static'
app = Flask(__name__, template_folder=TEMPLATE_FOLDER, static_folder = STATIC_FOLDER)
run_with_ngrok(app)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chat') 
def chatbot():
    return render_template('chat.html')

@app.route('/get')
def get_bot_response():
    userText = request.args.get('a')
    return answer_system(userText)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://1a29-34-75-195-104.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/images/photo-1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/js/slideshow.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/images/photo-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/images/photo-3.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/images/img-1.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/images/img-2.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/images/img-3.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] "GET /static/images/sns-1.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:20:38] 

안녕하세요, 무엇을 도와드릴까요?


...

INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:23:51] "GET /get?a=맛집%20알려줘 HTTP/1.1" 200 -


안녕하세요, 무엇을 도와드릴까요?


INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:23:57] "GET /get?a=카페%20ㄱㄱ HTTP/1.1" 200 -


안녕하세요, 무엇을 도와드릴까요?


INFO:werkzeug:127.0.0.1 - - [01/Oct/2022 10:24:16] "GET / HTTP/1.1" 200 -
